# A simple background Markov model in Pyro

It's not clear how pyro.plate function specifies conditional independence. I have 2 competing hypotheses for how it works:
1. (Null) If we didn't separate various latent/observed variables using pyro.plate, AND did't specify in the guide the dependence of given variable on a previously sampled one, pyro won't automatically and explicitly assume some kind of dependence! All pyro.plate does is it will simplify sampling and various other calculations the model performs when the plate screams INDEPENDENT!
2. (Alternate) By some mechanism I'm not aware of, pyro will construct will construct some conditional dependence on the variables not specified by a pyro.plate as independent! (How's that even possible if the involved variables are potentially real and unbounded?)

In [1]:
import numpy as np
import torch
from torch import nn
import pyro
from pyro import distributions as dist
import pandas as pd
from IPython.display import display

First, we'll generate some DNA data from just a second order Markov model without assuming any motifs or modules. Here, we'll choose the Markov model in a randomly generated fashion:

In [2]:
prior = dist.Dirichlet(torch.tensor([1., 1., 1., 1.])) # type: ignore
# Distribution from which Markov probabilities will be drawn from
phi = prior.sample([4, 4])
print(phi.shape)
# "phi" describes a second order Markov distribution over DNA base pairs
# for instance, phi[2, 3, 0] means probability of 0th BP conditioned on
# BPs 2 and 3 to appear before it

torch.Size([4, 4, 4])


Let's look at the various conditional probabilities that phi represents:

In [6]:
rows = ['(0, 0)', '(0, 1)', '(0, 2)', '(0, 3)',
        '(1, 0)', '(1, 1)', '(1, 2)', '(1, 3)',
        '(2, 0)', '(2, 1)', '(2, 2)', '(2, 3)',
        '(3, 0)', '(3, 1)', '(3, 2)', '(3, 3)']

flat_phi = np.reshape(phi, [-1, 4]) #

df_phi = pd.DataFrame({'Conditioned BP': rows, 
                   'P(0)': flat_phi[:, 0],
                   'P(1)': flat_phi[:, 1],
                   'P(2)': flat_phi[:, 2],
                   'P(3)': flat_phi[:, 3]})

In [7]:
display(df_phi)

,Conditioned BP,P(0),P(1),P(2),P(3)
0,"(0, 0)",0.102274,0.265847,0.278432,0.353446
1,"(0, 1)",0.035241,0.013038,0.738283,0.213438
2,"(0, 2)",0.068244,0.469061,0.250197,0.212499
3,"(0, 3)",0.190408,0.320309,0.394577,0.094706
4,"(1, 0)",0.130510,0.691025,0.059438,0.119028
5,"(1, 1)",0.182082,0.208129,0.075353,0.534436
6,"(1, 2)",0.893628,0.052813,0.004233,0.049326
7,"(1, 3)",0.027103,0.150291,0.240775,0.581830
8,"(2, 0)",0.066440,0.190860,0.099284,0.643416
9,"(2, 1)",0.258359,0.673523,0.039847,0.028271


For a cell in the above table, if the left column "Conditioned BP" has value (i,j) and the top row P(k), then the value in the cell represents this conditional probability: $P(X_u=k| X_{u-2}=i, X_{u-i}=j)$ for a position u in a DNA sequence 

Defining some constants

In [23]:
n = 100 # Length of DNA sequences
batch = 50 # Size of batch

Simulating a batch of DNA sequences based on the Markov model

In [24]:
X = np.empty((batch, n), dtype=int)
g = np.random.default_rng()
phi2 = phi.numpy() #Converted to NumPy array

for i in range(batch):
    prob = np.mean(phi2, axis=(0, 1))
    X[i, 0] = g.choice(4, p=prob) #Marginalizing for positions 1,2
    prob = np.mean(phi2, axis=0)[X[i, 0], :]
    X[i, 1] = g.choice(4, p=prob)
    for m in range(2, n):
        prob = phi2[X[i, m-2], X[i, m-1], :]
        X[i, m] = g.choice(4, p=prob)  # Declare X as torch tensor

X = torch.from_numpy(X) # shape: (100, 50)

In [17]:
#Looking at contents of X in the sequence at batch position 1
print(X[1, :])

tensor([2, 0, 3, 0, 3, 0, 3, 0, 0, 2, 1, 1, 1, 3, 3, 2, 0, 3, 2, 3, 2, 3, 1, 2,
        0, 3, 2, 0, 3, 1, 1, 1, 1, 3, 3, 2, 0, 3, 1, 1, 0, 1, 3, 3, 2, 0, 3, 2,
        3, 1, 2, 0, 2, 2, 0, 3, 0, 3, 2, 0, 3, 2, 0, 3, 1, 1, 3, 3, 0, 3, 2, 2,
        0, 3, 1, 1, 3, 0, 0, 0, 3, 0, 3, 1, 2, 0, 2, 1, 1, 3, 3, 2, 3, 1, 1, 1,
        1, 3, 2, 0])


With the simulated dataset, we will use a SVI model in Pyro to model the background Markov model (for X[:, m]) as a neural network that takes X[:, m-2] and X[:, m-2] as inputs! Let's define the neural network, a sequential module:

In [4]:
def Generate_NN(depth=3, width=16): # Input- (1,2,4)
    layers = []
    assert depth >= 0
    layers.append(nn.Flatten())
    if depth==0:
        layers.append(nn.Linear(8, 4))
    elif depth>0:
        layers.append(nn.Linear(8, width))
        layers.append(nn.ReLU())
        for _ in range(depth-2):
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(width, 4))
    
    layers.append(nn.Softmax(dim=-1))
    return nn.Sequential(*layers) #Output- (4,)

markov_nn = Generate_NN()

In [33]:
def model(X):
    MarkovNN = pyro.module("Neural Markov Model", markov_nn)
    X_oh = nn.functional.one_hot(X, 4).float() #type: ignore
    # One hot categorical encoding of the sequences
    X_oh = torch.concat([torch.zeros(50, 2, 4), X_oh], dim=1).float()
    # Padding each sequence with zeros in the beginning for unbiased Merkov
    # estimation
    for u in pyro.markov(range(X.shape[1]), history=2): #type:ignore
        probs = MarkovNN(torch.concat([X_oh[:, u, :], X_oh[:, u+1, :]], dim=1))
        # We make a vectorized plate over the batch dimension
        with pyro.plate("Batch"):
            pyro.sample("X_{}".format(u), dist.Categorical(probs), obs=X[:, u]) #type: ignore

Since there are no (random) latent variables defined in the function, there's the guide function in empty. The opimizer's job will be to optimize the neural network's weights in the model alone.

In [34]:
# An empty guide function
def guide(X):
    pass

Let's train the parameters of the neural network defined inside model function using a simple ELBO loss. The model is reparameterizable since there are no latent variables defined:

In [35]:
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO

In [36]:
def train(data, steps=10):
    # Clear out other trainable parameters in the current REPL/ipython kernel
    # session
    pyro.clear_param_store()
    opt = torch.optim.Adam
    scheduler = optim.StepLR({"optimizer": opt, "step_size": 2000, "gamma": 0.2, #type: ignore
                              "optim_args": {"lr": 0.001, "betas": (0.9, 0.999)}})
    svi = SVI(model, guide, scheduler, loss=Trace_ELBO())
    
    for k in range(steps):
        svi.step(data)
        scheduler.step()

In [37]:
train(X, 8000)

Let's verify how well the neural network has encoded the background Markov model. So we'll verify how well the netowrk recovers the conditional variable phi by training on the dataset X (which was generated using phi).

We'll generate a dataframe that encodes various such conditional probabilies derived from markov_nn:

In [14]:
inputs = torch.tensor([[0,0], [0,1], [0,2], [0,3],
                       [1,0], [1,1], [1,2], [1,3],
                       [2,0], [2,1], [2,2], [2,3],
                       [3,0], [3,1], [3,2], [3,3]]) # shape: (16,2)
inputs = nn.functional.one_hot(inputs).float() #type: ignore
out = markov_nn(inputs).detach().numpy() # (16, 4)

In [21]:
rows = ['(0, 0)', '(0, 1)', '(0, 2)', '(0, 3)',
        '(1, 0)', '(1, 1)', '(1, 2)', '(1, 3)',
        '(2, 0)', '(2, 1)', '(2, 2)', '(2, 3)',
        '(3, 0)', '(3, 1)', '(3, 2)', '(3, 3)']

df_nn = pd.DataFrame({'Conditioned BP': rows,
                      'P(0)': out[:, 0],
                      'P(1)': out[:, 1],
                      'P(2)': out[:, 2],
                      'P(3)': out[:, 3]})


In [22]:
df_nn

,Conditioned BP,P(0),P(1),P(2),P(3)
0,"(0, 0)",0.137307,0.296920,0.246492,0.319281
1,"(0, 1)",0.025741,0.012840,0.762165,0.199254
2,"(0, 2)",0.044388,0.490052,0.264570,0.200990
3,"(0, 3)",0.203108,0.307822,0.404627,0.084443
4,"(1, 0)",0.099135,0.739707,0.040597,0.120561
5,"(1, 1)",0.217215,0.208984,0.094421,0.479380
6,"(1, 2)",0.886130,0.055619,0.007972,0.050279
7,"(1, 3)",0.033181,0.129096,0.224584,0.613138
8,"(2, 0)",0.064364,0.194463,0.099152,0.642021
9,"(2, 1)",0.242307,0.684129,0.037053,0.036511


We'll also tabulate phi (once again), to compare with the neural network-generated values above:

In [8]:
display(df_phi)

,Conditioned BP,P(0),P(1),P(2),P(3)
0,"(0, 0)",0.102274,0.265847,0.278432,0.353446
1,"(0, 1)",0.035241,0.013038,0.738283,0.213438
2,"(0, 2)",0.068244,0.469061,0.250197,0.212499
3,"(0, 3)",0.190408,0.320309,0.394577,0.094706
4,"(1, 0)",0.130510,0.691025,0.059438,0.119028
5,"(1, 1)",0.182082,0.208129,0.075353,0.534436
6,"(1, 2)",0.893628,0.052813,0.004233,0.049326
7,"(1, 3)",0.027103,0.150291,0.240775,0.581830
8,"(2, 0)",0.066440,0.190860,0.099284,0.643416
9,"(2, 1)",0.258359,0.673523,0.039847,0.028271


We can see that there's a rough agreement between the two datasets df_nn and df_phi. We'll try to improve the accuracy of predictions by subsampling on a larger dataset X in the next section.

In [33]:
torch.save(markov_nn.state_dict(), "saved_nn.pt")
torch.save(phi, "saved_phi.pt")

## Subsampling

There's a discrepancy so far in that we chose the batch size to be small at 50 and repeatedly used the same data X in *every* training step! This is more like a global training than batchwise, which is actually the preferred way to train neural networks. So we will use a larger number of sequences for simulated the entire dataset (say 1000) and subsample a batch every time the model function is called:

In [8]:
import numpy as np
import torch
from torch import nn
import pyro
from pyro import distributions as dist
import pandas as pd
from IPython.display import display

import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO

In [2]:
def Generate_NN(depth=3, width=16):
    layers = []
    assert depth >= 0
    layers.append(nn.Flatten())
    if depth == 0:
        layers.append(nn.Linear(8, 4))
    elif depth > 0:
        layers.append(nn.Linear(8, width))
        layers.append(nn.ReLU())
        for _ in range(depth-2):
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(width, 4))

    layers.append(nn.Softmax(dim=-1))
    return nn.Sequential(*layers)


markov_nn = Generate_NN()

Here, we load the saved values of trained model (not extremely vital- one can simply use an untrained model too, without much loss!) from the previous section and the variable phi (that contains the conditional probabilities):

In [4]:
markov_nn.load_state_dict(torch.load("saved_nn.pt"))
phi = torch.load("saved_phi.pt")

We'll simulate a larger DNA dataset X more sequences inside a batch:

In [5]:
batch = 1000
n = 100

X = np.empty((batch, n), dtype=int)
g = np.random.default_rng()
phi2 = phi.numpy()  # Converted to NumPy array

for i in range(batch):
    prob = np.mean(phi2, axis=(0, 1))
    X[i, 0] = g.choice(4, p=prob)  # Marginalizing for positions 1,2
    prob = np.mean(phi2, axis=0)[X[i, 0], :]
    X[i, 1] = g.choice(4, p=prob)
    for m in range(2, n):
        prob = phi2[X[i, m-2], X[i, m-1], :]
        X[i, m] = g.choice(4, p=prob)

X = torch.from_numpy(X)  # Declare X as torch tensor

We'll redefine the model function to sample a sub-batch (of size 50 as before) of the dataset instead of the entire dataset like we did in the previous section!

In [23]:
def model(X):
    MarkovNN = pyro.module("Neural Markov Model", markov_nn)
    with pyro.plate("Batch", size=1000, subsample_size=50) as ind:
    # This randomly selects 50 indices among the 1000 possible ones
        X_oh = nn.functional.one_hot(X, 4).float().index_select(0, ind) #type: ignore
        # One hot categorical encoding of the sequences
        X_oh = torch.concat([torch.zeros(50, 2, 4), X_oh], dim=1).float()
        # Padding each sequence with zeros in the beginning for unbiased Merkov
        # estimation
        for u in pyro.markov(range(X.shape[1]), history=2): #type:ignore
            probs = MarkovNN(torch.concat([X_oh[:, u, :], X_oh[:, u+1, :]], dim=1))
            pyro.sample("X_{}".format(u), dist.Categorical(probs),  # type: ignore
                        obs=X.index_select(0, ind)[:, u])

As well as an empty guide function and the usual train function:

In [11]:
def guide(X):
    pass

def train(data, steps=10):
    # Clear out other trainable parameters in the current REPL/ipython kernel
    # session
    pyro.clear_param_store()
    opt = torch.optim.Adam
    scheduler = optim.StepLR({"optimizer": opt, "step_size": 1000, "gamma": 0.2,  # type: ignore
                              "optim_args": {"lr": 0.001, "betas": (0.9, 0.999)}})
    svi = SVI(model, guide, scheduler, loss=Trace_ELBO())

    for k in range(steps):
        svi.step(data)
        scheduler.step()

In [25]:
train(X, 4000)

Create and display dataframes for phi and markov_nn for comparison:

In [26]:
rows = ['(0, 0)', '(0, 1)', '(0, 2)', '(0, 3)',
        '(1, 0)', '(1, 1)', '(1, 2)', '(1, 3)',
        '(2, 0)', '(2, 1)', '(2, 2)', '(2, 3)',
        '(3, 0)', '(3, 1)', '(3, 2)', '(3, 3)']

flat_phi = np.reshape(phi, [-1, 4])

df_phi = pd.DataFrame({'Conditioned BP': rows,
                       'P(0)': flat_phi[:, 0],
                       'P(1)': flat_phi[:, 1],
                       'P(2)': flat_phi[:, 2],
                       'P(3)': flat_phi[:, 3]})

In [27]:
inputs = torch.tensor([[0, 0], [0, 1], [0, 2], [0, 3],
                       [1, 0], [1, 1], [1, 2], [1, 3],
                       [2, 0], [2, 1], [2, 2], [2, 3],
                       [3, 0], [3, 1], [3, 2], [3, 3]])
inputs = nn.functional.one_hot(inputs).float()  # type: ignore
out = markov_nn(inputs).detach().numpy()

df_nn = pd.DataFrame({'Conditioned BP': rows,
                      'P(0)': out[:, 0],
                      'P(1)': out[:, 1],
                      'P(2)': out[:, 2],
                      'P(3)': out[:, 3]})

In [28]:
print('Phi values')
display(df_phi)
print('Neural generated values')
display(df_nn)

Phi values


,Conditioned BP,P(0),P(1),P(2),P(3)
0,"(0, 0)",0.102274,0.265847,0.278432,0.353446
1,"(0, 1)",0.035241,0.013038,0.738283,0.213438
2,"(0, 2)",0.068244,0.469061,0.250197,0.212499
3,"(0, 3)",0.190408,0.320309,0.394577,0.094706
4,"(1, 0)",0.130510,0.691025,0.059438,0.119028
5,"(1, 1)",0.182082,0.208129,0.075353,0.534436
6,"(1, 2)",0.893628,0.052813,0.004233,0.049326
7,"(1, 3)",0.027103,0.150291,0.240775,0.581830
8,"(2, 0)",0.066440,0.190860,0.099284,0.643416
9,"(2, 1)",0.258359,0.673523,0.039847,0.028271


Neural generated values


,Conditioned BP,P(0),P(1),P(2),P(3)
0,"(0, 0)",0.100088,0.275475,0.273854,0.350582
1,"(0, 1)",0.032123,0.013971,0.742448,0.211458
2,"(0, 2)",0.064053,0.483263,0.227968,0.224715
3,"(0, 3)",0.185822,0.319798,0.398659,0.095720
4,"(1, 0)",0.127749,0.696895,0.056626,0.118730
5,"(1, 1)",0.188077,0.214259,0.066473,0.531191
6,"(1, 2)",0.891609,0.055109,0.006471,0.046811
7,"(1, 3)",0.025640,0.158575,0.234166,0.581619
8,"(2, 0)",0.063732,0.194204,0.100325,0.641739
9,"(2, 1)",0.263053,0.666568,0.036034,0.034345
